# About

## What is done in notebook:

1. Generate covisitation matrices for test set, that is matrices that contains information about what items are "clicked" together. Code for covisitation matrices is forked and modified from Chris Deotte's covistiation matrices from [here](https://www.kaggle.com/code/cdeotte/candidate-rerank-model-lb-0-575). Additional matrice is added for carts and constants used in matrices generatation are adjusted.

2. Generating 40 candidates for each session from test set, using covisitation matrices. While generating candidadates, "weights of similarity" between items are also saved, to be used as features in further models.

# Import

In [1]:
VER = 1

import pandas as pd, numpy as np
import os, sys, pickle, glob, gc
import cudf, itertools
import matplotlib.pyplot as plt
import datetime
import os
import subprocess, psutil
import datetime
import matplotlib.pyplot as plt
from time import sleep

# Config

In [3]:
class cfg:
    local = False
    track_gpu_memory = True
    load_in_cache = False

    print_steps = True
    
    keep_amount = 40
    clip_amount_in_score_calc = 40
    
    weight_const1_carts = 0.1
#     weight_const1_carts = 1.0
    weight_const1_orders = 1.5
    
    add_most_common = False # very low benefit
    
    recreate_matrices = True
    save_matrices = True
    mat_keep_amount = 35

## Files

In [4]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

def read_file(f):
    if cfg.load_in_cache:
        return cudf.DataFrame( data_cache[f] )
    else:
        return read_file_to_cudf(f)
        
def read_file_to_cudf(f):
    df = cudf.read_parquet(f)
    df.ts = (df.ts/1000).astype('int32')
    df['type'] = df['type'].map(type_labels).astype('int8')
    return df
def read_file_to_cache(f):
    df = pd.read_parquet(f)
    df.ts = (df.ts/1000).astype('int32')
    df['type'] = df['type'].map(type_labels).astype('int8')
    return df

In [5]:
%%time

# CACHE THE DATA ON CPU BEFORE PROCESSING ON GPU
data_cache = {}

if cfg.local:
    files = glob.glob('../input/otto-validation/*_parquet/*')
else:
    files = glob.glob('../input/otto-chunk-data-inparquet-format/*_parquet/*')
    test_files = glob.glob('../input/otto-chunk-data-inparquet-format/test_parquet/*')
    
if cfg.load_in_cache:
    for f in files: data_cache[f] = read_file_to_cache(f)

# CHUNK PARAMETERS
READ_CT = 5
DIVIDE_BY = 6
CHUNK = int( np.ceil( len(files)/DIVIDE_BY ))
print(f'We will process {len(files)} files, in groups of {READ_CT} and chunks of {CHUNK}.')

We will process 146 files, in groups of 5 and chunks of 25.
CPU times: user 1.03 ms, sys: 1.01 ms, total: 2.04 ms
Wall time: 29 ms


In [6]:
if cfg.recreate_matrices:
    files_for_top_20_clicks = []
    files_for_top_20_buys = []
    files_for_top_20_buy2buy = []
    files_for_click2cart = []
else:
    files_for_top_20_clicks = glob.glob('/kaggle/input/covisitation-to-test-candidates-dataset/top_20_clicks_*')
    files_for_top_20_buys = glob.glob('/kaggle/input/covisitation-to-test-candidates-dataset/top_15_carts_orders_*')
    files_for_top_20_buy2buy = glob.glob('/kaggle/input/covisitation-to-test-candidates-dataset/top_15_buy2buy_*')
    files_for_click2cart = glob.glob('/kaggle/input/covisitation-to-test-candidates-dataset/top_click2cart_v*')

print(len(files_for_top_20_clicks))
print(len(files_for_top_20_buys))
print(len(files_for_top_20_buy2buy))
print(len(files_for_click2cart))

0
0
0
0


# Helpers

In [7]:
def ts_format(ts):
    return datetime.datetime.fromtimestamp(ts).strftime("%b %d %Y  %H:%M:%S")
def print_date_ts(ts, end='\n'):
    print(ts_format(ts), end=end)
def print_date(d, end='\n'):
    print(d.strftime("%b %d %Y  %H:%M:%S"), end=end)
def mils_format(x):
    return f'{x:,}'
def len_mils(x):
    return mils_format(len(x))
def preview_df(df, head_show=1):
    print(len_mils(df))
    display(df.head(head_show))

In [8]:
# assert False

## Track memory

In [ ]:
gpu_available = None

try:
    subprocess.check_output('nvidia-smi')
    gpu_available = True

    print('Nvidia GPU detected!')
except Exception:
    gpu_available = False
    cfg.track_gpu_mem = False

    print('No Nvidia GPU in system!')
    
if gpu_available:
    dev0 = cudf.cupy.cuda.Device(0)
    def print_gpu_mem_info():
        free_memory, total_memory = dev0.mem_info
        print('Free -', free_memory // 1024**2, 'MiB', '\tTotal -', total_memory // 1024**2, 'MiB')
    print_gpu_mem_info()
    
def clear_gpu_log():
    with open('./gpu-report.csv', 'w') as file:
        cols = [
            'date', 'used_MiB'
        ]
        line = ','.join(cols) + '\n'
        file.write(line)

def start_logging_gpu_info(every='2'):
    log_gpu_subprocess = subprocess.Popen(
        ["watch","-n",every, "./track-gpu.sh"],
        stdout=subprocess.PIPE
    )
    
    return log_gpu_subprocess

def stop_logging_gpu_info():
    for proc in psutil.process_iter():
        if 'watch' in proc.name() and './track-gpu.sh' in proc.cmdline():
            proc.kill()

def plot_gpu_usage(mins_to_plot=2, interval=2):
    if not os.path.exists('./gpu-report.csv'):
        return
        
    df = pd.read_csv('./gpu-report.csv')

    entries_num = int(mins_to_plot * 60 // interval)
    df = df[-entries_num:]
    df['date'] = df.date.apply(lambda d: datetime.datetime.fromisoformat(d))

    plt.figure(figsize=(12,4))
    plt.title(f'used GPU Memory')
    plt.plot(df.date, df['used_MiB'], color='blue')
    plt.show()

def create_file_with_logging_command():
    with open('./track-gpu.sh', 'w') as file:
        file.write(
            'values=$(nvidia-smi | sed -n \'10p\' | awk \'{print $9}\' | sed -nz \'s/MiB\\n//gp\'); echo $(date +%Y-%m-%d#%H:%M:%S),$(($values)) >> ' + f'{os.getcwd()}/gpu-report.csv'
        )

def run_logging_file():
    subprocess.run(
        [f'{os.getcwd()}/track-gpu.sh'],
        shell=True
    )

def allow_logging_file_execution():
    os.system('chmod a+x ./track-gpu.sh')
    
if cfg.track_gpu_mem:
    create_file_with_logging_command()
    allow_logging_file_execution()

    stop_logging_gpu_info()
    clear_gpu_log()
    run_logging_file() # to add one memory value
    
    watch_process = start_logging_gpu_info()
    
    df = pd.read_csv('./gpu-report.csv')
    df.tail()
    
if cfg.track_gpu_mem:
    sleep(2)
    plot_gpu_usage()

# Step 1 - Compute Co-visitation Matrices

We will compute 4 co-visitation matrices using RAPIDS cuDF on GPU. This is 30x faster than using Pandas CPU like other public notebooks! For maximum speed, set the variable `DISK_PIECES` to the smallest number possible based on the GPU you are using without incurring memory errors. If you run this code offline with 32GB GPU ram, then you can use `DISK_PIECES = 1` and compute each co-visitation matrix in almost 1 minute! Kaggle's GPU only has 16GB ram, so we use `DISK_PIECES = 4` and it takes an amazing 3 minutes each! Below are some of the tricks to speed up computation
* Use RAPIDS cuDF GPU instead of Pandas CPU
* Read disk once and save in CPU RAM for later GPU multiple use
* Process largest amount of data possible on GPU at one time
* Merge data in two stages. Multiple small to single medium. Multiple medium to single large.
* Write result as parquet instead of dictionary

## 1) "Carts Orders" Co-visitation Matrix - Type Weighted

In [9]:
%%time
# files_for_top_20_buys = []
if len(files_for_top_20_buys) == 0:
    type_weight = {0:1, 1:6, 2:3}

    # USE SMALLEST DISK_PIECES POSSIBLE WITHOUT MEMORY ERROR
    DISK_PIECES = 4
    SIZE = 1.86e6/DISK_PIECES

    # COMPUTE IN PARTS FOR MEMORY MANGEMENT
    for PART in range(DISK_PIECES):
        print()
        print('### DISK PART',PART+1)

        # MERGE IS FASTEST PROCESSING CHUNKS WITHIN CHUNKS
        # => OUTER CHUNKS
        for j in range(DIVIDE_BY):
            a = j*CHUNK
            b = min( (j+1)*CHUNK, len(files) )
            print(f'Processing files {a} thru {b-1} in groups of {READ_CT}...', end=' ')

            # => INNER CHUNKS
            for k in range(a,b,READ_CT):
                # READ FILE
                df = [read_file(files[k])]
                for i in range(1,READ_CT): 
                    if k+i<b: df.append( read_file(files[k+i]) )
                df = cudf.concat(df,ignore_index=True,axis=0)
                df = df.sort_values(['session','ts'],ascending=[True,False])
                # USE TAIL OF SESSION
                df = df.reset_index(drop=True)
                df['n'] = df.groupby('session').cumcount()
                df = df.loc[df.n<30].drop('n',axis=1)
                # CREATE PAIRS
                df = df.merge(df,on='session')
                df = df.loc[ ((df.ts_x - df.ts_y).abs()< 30 * 60) & (df.aid_x != df.aid_y) ]
                # MEMORY MANAGEMENT COMPUTE IN PARTS
                df = df.loc[(df.aid_x >= PART*SIZE)&(df.aid_x < (PART+1)*SIZE)]
                # ASSIGN WEIGHTS
                df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
                df['wgt'] = df.type_y.map(type_weight)
                df = df[['aid_x','aid_y','wgt']]
                df.wgt = df.wgt.astype('float32')
                df = df.groupby(['aid_x','aid_y']).wgt.sum()
                # COMBINE INNER CHUNKS
                if k==a: tmp2 = df
                else: tmp2 = tmp2.add(df, fill_value=0)
                print(k,', ',end='')
            print()
            # COMBINE OUTER CHUNKS
            if a==0: tmp = tmp2
            else: tmp = tmp.add(tmp2, fill_value=0)
            del tmp2, df
            gc.collect()
        # CONVERT MATRIX TO DICTIONARY
        tmp = tmp.reset_index()
        tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
        # SAVE TOP 40
        tmp = tmp.reset_index(drop=True)
        tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
        tmp = tmp.loc[tmp.n < cfg.mat_keep_amount].drop('n',axis=1)
        # SAVE PART TO DISK (convert to pandas first uses less memory)

        tmp = tmp.to_pandas()
        tmp.wgt = np.log(1.73+tmp.wgt)
        tmp.to_parquet(f'top_15_carts_orders_v{VER}_{PART}.pqt')

        files_for_top_20_buys = glob.glob('top_15_carts_orders_v*')

        # tmp.to_pandas().to_parquet(f'top_15_carts_orders_v{VER}_{PART}.pqt')
    del tmp
print(files_for_top_20_buys)


### DISK PART 1
Processing files 0 thru 24 in groups of 5... 

/opt/conda/lib/python3.7/site-packages/cudf/core/frame.py:2600: UserWarning: When using a sequence of booleans for `ascending`, `na_position` flag is not yet supported and defaults to treating nulls as greater than all numbers
  "When using a sequence of booleans for `ascending`, "


0 , 5 , 10 , 15 , 20 , 
Processing files 25 thru 49 in groups of 5... 25 , 30 , 35 , 40 , 45 , 
Processing files 50 thru 74 in groups of 5... 50 , 55 , 60 , 65 , 70 , 
Processing files 75 thru 99 in groups of 5... 75 , 80 , 85 , 90 , 95 , 
Processing files 100 thru 124 in groups of 5... 100 , 105 , 110 , 115 , 120 , 
Processing files 125 thru 145 in groups of 5... 125 , 130 , 135 , 140 , 145 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 5... 0 , 5 , 10 , 15 , 20 , 
Processing files 25 thru 49 in groups of 5... 25 , 30 , 35 , 40 , 45 , 
Processing files 50 thru 74 in groups of 5... 50 , 55 , 60 , 65 , 70 , 
Processing files 75 thru 99 in groups of 5... 75 , 80 , 85 , 90 , 95 , 
Processing files 100 thru 124 in groups of 5... 100 , 105 , 110 , 115 , 120 , 
Processing files 125 thru 145 in groups of 5... 125 , 130 , 135 , 140 , 145 , 

### DISK PART 3
Processing files 0 thru 24 in groups of 5... 0 , 5 , 10 , 15 , 20 , 
Processing files 25 thru 49 in groups of 5... 25 , 30 , 

In [10]:
if cfg.save_matrices:
    for f in files_for_top_20_buys:
        tmp = cudf.read_parquet(f)
        file_name = os.path.basename(f)
        print(file_name)
        tmp.to_parquet(file_name)

top_15_carts_orders_v6_0.pqt
top_15_carts_orders_v6_2.pqt
top_15_carts_orders_v6_1.pqt
top_15_carts_orders_v6_3.pqt


## 2) "Buy2Buy" Co-visitation Matrix

In [11]:
%%time
# files_for_top_20_buy2buy = []
if len(files_for_top_20_buy2buy) == 0:

    # USE SMALLEST DISK_PIECES POSSIBLE WITHOUT MEMORY ERROR
    DISK_PIECES = 1
    SIZE = 1.86e6/DISK_PIECES
    DIVIDE_BY = 5
    READ_CT = 5
    CHUNK = int( np.ceil( len(files)/DIVIDE_BY ))
    print(f'We will process {len(files)} files, in groups of {READ_CT} and chunks of {CHUNK}.')

    # COMPUTE IN PARTS FOR MEMORY MANGEMENT
    for PART in range(DISK_PIECES):
        print()
        print('### DISK PART',PART+1)

        # MERGE IS FASTEST PROCESSING CHUNKS WITHIN CHUNKS
        # => OUTER CHUNKS
        for j in range(DIVIDE_BY):
            a = j*CHUNK
            b = min( (j+1)*CHUNK, len(files) )
            print(f'Processing files {a} thru {b-1} in groups of {READ_CT}...')

            # => INNER CHUNKS
            for k in range(a,b,READ_CT):
                # READ FILE
                df = [read_file(files[k])]
                print(k, end=' ')
                for i in range(1,READ_CT): 
                    if k+i<b:
                        print(k+i, end=' ')
                        df.append( read_file(files[k+i]) )

                df = cudf.concat(df,ignore_index=True,axis=0)
                df = df.loc[df['type'].isin([1,2])] # ONLY WANT CARTS AND ORDERS
                df = df.sort_values(['session','ts'],ascending=[True,False])
                # USE TAIL OF SESSION
                df = df.reset_index(drop=True)
                df['n'] = df.groupby('session').cumcount()
                df = df.loc[df.n<20].drop('n',axis=1)
                # CREATE PAIRS
                df = df.merge(df,on='session')
                df = df.loc[ ((df.ts_x - df.ts_y).abs()< 14 * 24 * 60 * 60) & (df.aid_x != df.aid_y) ] # 14 DAYS
                # MEMORY MANAGEMENT COMPUTE IN PARTS
                df = df.loc[(df.aid_x >= PART*SIZE)&(df.aid_x < (PART+1)*SIZE)]
                # ASSIGN WEIGHTS

                # df = df[['session', 'aid_x', 'aid_y','ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y'])
                # df['wgt'] = 1 + 3*(df.ts_x - ts_min_used)/(ts_max_used-ts_min_used)

                df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
                df['wgt'] = 1

                df = df[['aid_x','aid_y','wgt']]
                df.wgt = df.wgt.astype('float32')
                df = df.groupby(['aid_x','aid_y']).wgt.sum()


                # COMBINE INNER CHUNKS
                if k==a: tmp2 = df
                else: tmp2 = tmp2.add(df, fill_value=0)
                # print(k,', ',end='')
            print()
            # COMBINE OUTER CHUNKS
            if a==0: tmp = tmp2
            else: tmp = tmp.add(tmp2, fill_value=0)
            del tmp2, df
            gc.collect()
        # CONVERT MATRIX TO DICTIONARY
        tmp = tmp.reset_index()
        tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
        # SAVE TOP 40
        tmp = tmp.reset_index(drop=True)
        tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
        tmp = tmp.loc[tmp.n < cfg.mat_keep_amount].drop('n',axis=1)
        # SAVE PART TO DISK (convert to pandas first uses less memory)

        tmp = tmp.to_pandas()
        tmp.wgt = np.log(1.73+tmp.wgt)
        tmp.to_parquet(f'top_15_buy2buy_v{VER}_{PART}.pqt')

        # tmp.to_pandas().to_parquet(f'top_15_buy2buy_v{VER}_{PART}.pqt')
    del tmp
    files_for_top_20_buy2buy = glob.glob(f'top_15_buy2buy_v{VER}*')

    # Carts and orders occuring together:

    top_20_buy2buy_df = cudf.DataFrame()
    for f in files_for_top_20_buy2buy:
        print(f)
        top_20_buy2buy_df = cudf.concat([top_20_buy2buy_df, cudf.read_parquet(f)])
    preview_df(top_20_buy2buy_df)
    
print(files_for_top_20_buy2buy)

# 1,314,327

We will process 146 files, in groups of 5 and chunks of 30.

### DISK PART 1
Processing files 0 thru 29 in groups of 5...
0 1 2 3 4 5 6 7 8 9 

/opt/conda/lib/python3.7/site-packages/cudf/core/frame.py:2600: UserWarning: When using a sequence of booleans for `ascending`, `na_position` flag is not yet supported and defaults to treating nulls as greater than all numbers
  "When using a sequence of booleans for `ascending`, "


10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
Processing files 30 thru 59 in groups of 5...
30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
Processing files 60 thru 89 in groups of 5...
60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 
Processing files 90 thru 119 in groups of 5...
90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 
Processing files 120 thru 145 in groups of 5...
120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 
top_15_buy2buy_v6_0.pqt
21,489,751


,aid_x,aid_y,wgt
0,1,279811,1.004302


['top_15_buy2buy_v6_0.pqt']
CPU times: user 42.5 s, sys: 10.5 s, total: 53.1 s
Wall time: 54.4 s


In [12]:
if cfg.save_matrices:
    for f in files_for_top_20_buy2buy:
        tmp = cudf.read_parquet(f)
        file_name = os.path.basename(f)
        print(file_name)
        tmp.to_parquet(file_name)

top_15_buy2buy_v6_0.pqt


## 3) "Clicks" Co-visitation Matrix - Time Weighted

In [13]:
%%time
# files_for_top_20_clicks = []
if len(files_for_top_20_clicks) == 0:

    # USE SMALLEST DISK_PIECES POSSIBLE WITHOUT MEMORY ERROR
    DISK_PIECES = 4
    SIZE = 1.86e6/DISK_PIECES

    # COMPUTE IN PARTS FOR MEMORY MANGEMENT
    for PART in range(DISK_PIECES):
        print()
        print('### DISK PART',PART+1)

        # MERGE IS FASTEST PROCESSING CHUNKS WITHIN CHUNKS
        # => OUTER CHUNKS
        for j in range(DIVIDE_BY):
            a = j*CHUNK
            b = min( (j+1)*CHUNK, len(files) )
            print(f'Processing files {a} thru {b-1} in groups of {READ_CT}...', end=' ')

            # => INNER CHUNKS
            for k in range(a,b,READ_CT):
                print(k,', ',end='')
                # READ FILE
                df = [read_file(files[k])]
                for i in range(1,READ_CT): 
                    if k+i<b: 
                        print(k+i, end=' ')
                        df.append( read_file(files[k+i]) )
                        
                df = cudf.concat(df,ignore_index=True,axis=0)
                df = df.sort_values(['session','ts'],ascending=[True,False])
                # USE TAIL OF SESSION
                df = df.reset_index(drop=True)
                df['n'] = df.groupby('session').cumcount()
                df = df.loc[df.n<30].drop('n',axis=1)
                # CREATE PAIRS
                df = df.merge(df,on='session')
                df = df.loc[ ((df.ts_x - df.ts_y).abs()< 30 * 60) & (df.aid_x != df.aid_y) ]
                # MEMORY MANAGEMENT COMPUTE IN PARTS
                df = df.loc[(df.aid_x >= PART*SIZE)&(df.aid_x < (PART+1)*SIZE)]
                # ASSIGN WEIGHTS
                df = df[['session', 'aid_x', 'aid_y','ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y'])
                df['wgt'] = 1 + 3*(df.ts_x - 1659304800)/(1662328791-1659304800)
                df = df[['aid_x','aid_y','wgt']]
                df.wgt = df.wgt.astype('float32')
                df = df.groupby(['aid_x','aid_y']).wgt.sum()
                # COMBINE INNER CHUNKS
                if k==a: tmp2 = df
                else: tmp2 = tmp2.add(df, fill_value=0)
#                 print(k,', ',end='')
            print()
            # COMBINE OUTER CHUNKS
            if a==0: tmp = tmp2
            else: tmp = tmp.add(tmp2, fill_value=0)
            del tmp2, df
            gc.collect()
        # CONVERT MATRIX TO DICTIONARY
        tmp = tmp.reset_index()
        tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
        # SAVE TOP 40
        tmp = tmp.reset_index(drop=True)
        tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
        tmp = tmp.loc[tmp.n < cfg.mat_keep_amount].drop('n',axis=1)
        # SAVE PART TO DISK (convert to pandas first uses less memory)

        tmp = tmp.to_pandas()
        tmp.wgt = np.log(1.73+tmp.wgt)
        tmp.to_parquet(f'top_20_clicks_v{VER}_{PART}.pqt')

        # tmp.to_pandas().to_parquet(f'top_20_clicks_v{VER}_{PART}.pqt')
        
        files_for_top_20_clicks = glob.glob(f'top_20_clicks_v{VER}_*')
    del tmp
print(files_for_top_20_clicks)


### DISK PART 1
Processing files 0 thru 29 in groups of 5... 0 , 1 2 3 4 

/opt/conda/lib/python3.7/site-packages/cudf/core/frame.py:2600: UserWarning: When using a sequence of booleans for `ascending`, `na_position` flag is not yet supported and defaults to treating nulls as greater than all numbers
  "When using a sequence of booleans for `ascending`, "


5 , 6 7 8 9 10 , 11 12 13 14 15 , 16 17 18 19 20 , 21 22 23 24 25 , 26 27 28 29 
Processing files 30 thru 59 in groups of 5... 30 , 31 32 33 34 35 , 36 37 38 39 40 , 41 42 43 44 45 , 46 47 48 49 50 , 51 52 53 54 55 , 56 57 58 59 
Processing files 60 thru 89 in groups of 5... 60 , 61 62 63 64 65 , 66 67 68 69 70 , 71 72 73 74 75 , 76 77 78 79 80 , 81 82 83 84 85 , 86 87 88 89 
Processing files 90 thru 119 in groups of 5... 90 , 91 92 93 94 95 , 96 97 98 99 100 , 101 102 103 104 105 , 106 107 108 109 110 , 111 112 113 114 115 , 116 117 118 119 
Processing files 120 thru 145 in groups of 5... 120 , 121 122 123 124 125 , 126 127 128 129 130 , 131 132 133 134 135 , 136 137 138 139 140 , 141 142 143 144 145 , 

### DISK PART 2
Processing files 0 thru 29 in groups of 5... 0 , 1 2 3 4 5 , 6 7 8 9 10 , 11 12 13 14 15 , 16 17 18 19 20 , 21 22 23 24 25 , 26 27 28 29 
Processing files 30 thru 59 in groups of 5... 30 , 31 32 33 34 35 , 36 37 38 39 40 , 41 42 43 44 45 , 46 47 48 49 50 , 51 52 53 54 

In [14]:
if cfg.save_matrices:
    for f in files_for_top_20_clicks:
        tmp = cudf.read_parquet(f)
        file_name = os.path.basename(f)
        print(file_name)
        tmp.to_parquet(file_name)

top_20_clicks_v6_0.pqt
top_20_clicks_v6_2.pqt
top_20_clicks_v6_3.pqt
top_20_clicks_v6_1.pqt


## 4) Carts mat

In [15]:
%%time
# files_for_click2cart = []
if len(files_for_click2cart) == 0:

    # USE SMALLEST DISK_PIECES POSSIBLE WITHOUT MEMORY ERROR
    DISK_PIECES = 4
    SIZE = 1.86e6/DISK_PIECES
    READ_CT = 4
    DIVIDE_BY = 5

    # COMPUTE IN PARTS FOR MEMORY MANGEMENT
    for PART in range(DISK_PIECES):
        print()
        print('### DISK PART',PART+1)

        # MERGE IS FASTEST PROCESSING CHUNKS WITHIN CHUNKS
        # => OUTER CHUNKS
        for j in range(DIVIDE_BY):
            a = j*CHUNK
            b = min( (j+1)*CHUNK, len(files) )
            print(f'Processing files {a} thru {b-1} in groups of {READ_CT}...', end=' ')

            # => INNER CHUNKS
            for k in range(a,b,READ_CT):
                # READ FILE
                df = [read_file(files[k])]
                print(k, end=' ')
                for i in range(1,READ_CT): 
                    print(k+i, end=' ')
                    if k+i<b: df.append( read_file(files[k+i]) )
                df = cudf.concat(df,ignore_index=True,axis=0)

                df = df.loc[df['type'] < 2] # CLICKS and CARTS 
                df = df.sort_values(['session','ts'],ascending=[True,False])
                # USE TAIL OF SESSION
                df = df.reset_index(drop=True)
                df['n'] = df.groupby('session').cumcount()
                df = df.loc[df.n<20].drop('n',axis=1)

                # CREATE PAIRS
                df = df.merge(df,on='session')
                df = df.loc[ ((df.ts_x - df.ts_y).abs()< 14 * 24 * 60 * 60) & (df.aid_x != df.aid_y) ] # 14 DAYS

                # MEMORY MANAGEMENT COMPUTE IN PARTS
                df = df.loc[(df.aid_x >= PART*SIZE)&(df.aid_x < (PART+1)*SIZE)]

                # ASSIGN WEIGHTS

                # df = df[['session', 'aid_x', 'aid_y','ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y'])
                # df['wgt'] = 1 + 3*(df.ts_x - 1659304800)/(1662328791-1659304800)

                df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
                df['wgt'] = 1
                
                df = df[['aid_x','aid_y','wgt']]
                df.wgt = df.wgt.astype('float32')
                df = df.groupby(['aid_x','aid_y']).wgt.sum()
                
                # COMBINE INNER CHUNKS
                if k==a: tmp2 = df
                else: tmp2 = tmp2.add(df, fill_value=0)
                # print(k,'-',k+i,', ',end='')
            print()
            # COMBINE OUTER CHUNKS
            if a==0: tmp = tmp2
            else: tmp = tmp.add(tmp2, fill_value=0)
            del tmp2, df
            gc.collect()
        # CONVERT MATRIX TO DICTIONARY
        tmp = tmp.reset_index()
        tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
        # SAVE TOP 40
        tmp = tmp.reset_index(drop=True)
        tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
        tmp = tmp.loc[tmp.n < cfg.mat_keep_amount].drop('n',axis=1)
        # SAVE PART TO DISK (convert to pandas first uses less memory)
        
        tmp = tmp.to_pandas()
        tmp.wgt = np.log(1.73+tmp.wgt)
        tmp.to_parquet(f'top_click2cart_v{VER}_{PART}.pqt')

        # tmp.to_pandas().to_parquet(f'top_click2cart_v{VER}_{PART}.pqt')
    del tmp
    files_for_click2cart = glob.glob('top_click2cart_v*')

    # Carts and clicks occuring together:

    top_click2cart_df = cudf.DataFrame()
    for f in files_for_click2cart:
        print(f)
        top_click2cart_df = cudf.concat([top_click2cart_df, cudf.read_parquet(f)])
    preview_df(top_click2cart_df)

print(files_for_click2cart)


### DISK PART 1
Processing files 0 thru 29 in groups of 4... 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 
Processing files 30 thru 59 in groups of 4... 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 
Processing files 60 thru 89 in groups of 4... 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 
Processing files 90 thru 119 in groups of 4... 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 
Processing files 120 thru 145 in groups of 4... 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 

### DISK PART 2
Processing files 0 thru 29 in groups of 4... 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 
Processing files 30 thru 59 in groups of 4... 30 31 32 33 34 35 36 37 38 39 40 41 42 43 4

,aid_x,aid_y,wgt
0,0,532042,2.275214


['top_click2cart_v6_0.pqt', 'top_click2cart_v6_3.pqt', 'top_click2cart_v6_2.pqt', 'top_click2cart_v6_1.pqt']
CPU times: user 3min 52s, sys: 1min 22s, total: 5min 14s
Wall time: 5min 20s


In [16]:
if cfg.save_matrices:
    for f in files_for_click2cart:
        tmp = cudf.read_parquet(f)
        file_name = os.path.basename(f)
        print(file_name)
        tmp.to_parquet(file_name)

top_click2cart_v6_0.pqt
top_click2cart_v6_3.pqt
top_click2cart_v6_2.pqt
top_click2cart_v6_1.pqt


In [17]:
# FREE MEMORY
try:
    if cfg.load_in_cache:
        del data_cache
except Exception as e:
    print(e)
_ = gc.collect()

In [18]:
# assert False

# Step 2 - ReRank - choose candidates using handcrafted rules

In [19]:
# !ls -l

In [20]:
def load_test():    
    dfs = []
    for e, chunk_file in enumerate(test_files):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

test_df = load_test()
print('Test data has shape',test_df.shape)
test_df.head()

Test data has shape (6928123, 4)


,session,aid,ts,type
0,13099779,245308,1661795832,0
1,13099779,245308,1661795862,1
2,13099779,972319,1661795888,0
3,13099779,972319,1661795898,1
4,13099779,245308,1661795907,0


In [21]:
# TOP CLICKS AND ORDERS IN TEST

top_clicks = test_df.loc[test_df['type']==0,'aid'].value_counts().index.values[:cfg.keep_amount]
top_carts = test_df.loc[test_df['type']==1,'aid'].value_counts().index.values[:cfg.keep_amount]
top_orders = test_df.loc[test_df['type']==2,'aid'].value_counts().index.values[:cfg.keep_amount]

top_clicks_df = pd.DataFrame({'aid':top_clicks})
top_carts_df = pd.DataFrame({'aid':top_carts})
top_orders_df = pd.DataFrame({'aid':top_orders})

In [22]:
def load_top_20_clicks_df():
    top_20_clicks_df = cudf.DataFrame()
    files_for_top_20_clicks.sort()
    for f in files_for_top_20_clicks:
        print(f, end=' ')
        top_20_clicks_df = cudf.concat([top_20_clicks_df, cudf.read_parquet(f)])
    # preview_df(top_20_clicks_df)

    return top_20_clicks_df
def load_top_20_buys_df():
    top_20_buys_df = cudf.DataFrame()
    for f in files_for_top_20_buys:
        print(f, end=' ')
        top_20_buys_df = cudf.concat([top_20_buys_df, cudf.read_parquet(f)])
    # print()
    # preview_df(top_20_buys_df)

    return top_20_buys_df
def load_top_20_buy2buy_df():
    # Carts and orders occuring together:
    top_20_buy2buy_df = cudf.DataFrame()
    for f in files_for_top_20_buy2buy:
        print(f, end=' ')
        top_20_buy2buy_df = cudf.concat([top_20_buy2buy_df, cudf.read_parquet(f)])
    # preview_df(top_20_buy2buy_df)

    return top_20_buy2buy_df
def load_top_click2cart_df():
    # Carts and clicks occuring together:
    top_click2cart_df = cudf.DataFrame()
    for f in files_for_click2cart:
        print(f, end=' ')
        top_click2cart_df = cudf.concat([top_click2cart_df, cudf.read_parquet(f)])
    # preview_df(top_click2cart_df)
    return top_click2cart_df

In [23]:
def calc_event_recall(ev_preds_df, event_name, clip_amount=cfg.clip_amount_in_score_calc):
    weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}
    sub = ev_preds_df
    sub['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
    sub.labels = sub.labels.apply(lambda x: x[:clip_amount])
    
    test_labels = pd.read_parquet(test_labels_file)
    test_labels = test_labels.loc[test_labels['type']==event_name]
    test_labels = test_labels.merge(sub, how='left', on=['session'])

    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
    
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0, clip_amount)
    
    recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
    # score += weights[event_name]*recall
    print(f'{event_name} recall =',recall)

    return recall
    
def calc_score(event_name, clip_amount):
    ev_preds_df = {}
    for ev in [event_name]:
        files_for_ev = glob.glob(f'{ev}_pred_df*')
        # print(ev, len(files_for_ev))
        ev_preds_df[ev] = pd.DataFrame()
        for f in files_for_ev:
            ev_preds_df[ev] = pd.concat([ev_preds_df[ev], pd.read_parquet(f)], ignore_index=True)
        # preview_df(ev_preds_df[ev])

    return calc_event_recall(ev_preds_df[event_name], event_name, clip_amount)

In [24]:
def add_most_common(df, most_common, event_name, keep_amount=cfg.keep_amount):
    df = df.to_pandas()
    tmp1 = df[['session']].drop_duplicates('session').copy()
    tmp1 = pd.merge(tmp1, most_common, how='cross')
    if event_name=='clicks':
        tmp1['type'] = -1
        tmp1['repetitions'] = -1
        tmp1['clicks_similarity_wgt'] = -1
        tmp1[f'{event_name}_final_wgt'] = -1
    else:
        tmp1['type'] = -1
        tmp1['repetitions'] = -1
        tmp1['top_buys_wgt'] = -1
        tmp1['top_buy2buy_wgt'] = -1
        tmp1[f'{event_name}_final_wgt'] = -1

    df = pd.concat([df, tmp1]).sort_values(['session', f'{event_name}_final_wgt'], ascending=False)
    df = df.drop_duplicates(['session', 'aid'], ignore_index=True)
    df = df.sort_values(['session', f'{event_name}_final_wgt'], ascending=False).reset_index(drop=True)

    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n < keep_amount]
    del df['n']
    df = df.reset_index(drop=True)
    
    return df

In [25]:
def add_items_from_buys_matrices(df, for_event, keep_amount=cfg.keep_amount, weight_const1=1.0):
    aids1_df = df.loc[df.type != 0]
    # preview_df(aids1_df)

    # df = df.drop_duplicates(['session', 'aid'], ignore_index=True) # other ways of duplicates handling can be done
    df['repetitions'] = 1
    df = cudf.merge(
        df[['session', 'aid', 'type','ts']], 
        df.groupby(['session', 'aid', 'type']).repetitions.sum().reset_index(),
        on=['session', 'aid', 'type'],
        how='inner'
    )
    
    df = cudf.merge(
        df, 
        df.groupby('session').ts.max().reset_index().rename(columns={'ts': 'ts_max'}), 
        on='session', how='inner'
    )

    df['ts_distance'] = df['ts_max'] - df['ts']
    df['ts_distance'] = np.log(df['ts_distance']+1)
    df['ts_distance'] = df['ts_distance'].max()- df['ts_distance']
        
    # preview_df(df)

    # print('df',df.session.nunique())
    # Events at same time in session
    # some sessions will be lost, because inner merge and top_20_buys_df do not have all sessions
    # But adding df sessions at function end will fix sessions nunique to be same as test_df 
    aids2_df = cudf.merge(df[['session', 'aid']], top_20_buys_df, left_on='aid', right_on='aid_x', how='inner') 
    aids2_df.wgt = aids2_df.wgt.astype('float32')
#     aids2_df.wgt = np.log(1+aids2_df.wgt)
    aids2_df.loc[aids2_df.wgt > 200, 'wgt'] = 200

    
    # print('aids2', aids2_df.session.nunique())

    aids2_df = aids2_df.groupby(['session', 'aid_y']).wgt.sum().reset_index()
    aids2_df = aids2_df.sort_values(['session', 'wgt'], ascending=[True, False], ignore_index=True).reset_index(drop=True)
    aids2_df = aids2_df.rename(columns={'aid_y': 'aid', 'wgt': 'top_buys_wgt'})
    aids2_df['top_buy2buy_wgt'] = 0

    # preview_df(aids2_df,5)

    if for_event == 'carts':
        # Carts and clicks occuring together
        aids3_df = cudf.merge(df[['session', 'aid']], top_click2cart_df, left_on='aid', right_on='aid_x', how='inner')

        aids3_df = aids3_df.groupby(['session', 'aid_y']).wgt.sum().reset_index()
        aids3_df.wgt = aids3_df.wgt.astype('float32')
#         aids3_df.wgt = np.log(1+aids3_df.wgt)
        aids3_df.loc[aids3_df.wgt > 200, 'wgt'] = 200
        
        aids3_df = aids3_df.sort_values(['session', 'wgt'], ascending=[True, False], ignore_index=True).reset_index(drop=True)
        aids3_df = aids3_df.rename(columns={'aid_y': 'aid', 'wgt': 'top_buy2buy_wgt'})
        aids3_df['top_buys_wgt'] = 0
    else:
        # Carts and orders occuring together
        aids3_df = cudf.merge(df[['session', 'aid']], top_20_buy2buy_df, left_on='aid', right_on='aid_x', how='inner')

        aids3_df = aids3_df.groupby(['session', 'aid_y']).wgt.sum().reset_index()
        aids3_df.wgt = aids3_df.wgt.astype('float32')
#         aids3_df.wgt = np.log(1+aids3_df.wgt)
        aids3_df.loc[aids3_df.wgt > 200, 'wgt'] = 200
        
        aids3_df = aids3_df.sort_values(['session', 'wgt'], ascending=[True, False], ignore_index=True).reset_index(drop=True)
        aids3_df = aids3_df.rename(columns={'aid_y': 'aid', 'wgt': 'top_buy2buy_wgt'})
        aids3_df['top_buys_wgt'] = 0

    # preview_df(aids3_df,5)

    aids4_df = cudf.concat([aids2_df, aids3_df], ignore_index=True)
    aids4_df.top_buys_wgt = aids4_df.top_buys_wgt.astype('float32')
    aids4_df.top_buy2buy_wgt = aids4_df.top_buy2buy_wgt.astype('float32')

    # preview_df(aids4_df,5)

    aids4_df = aids4_df.groupby(['session', 'aid']).sum().reset_index()
    if for_event == 'carts':
        aids4_df['wgt'] = aids4_df['top_buys_wgt'] + weight_const1 * aids4_df['top_buy2buy_wgt']
    else:
        aids4_df['wgt'] = aids4_df['top_buys_wgt'] + weight_const1 * aids4_df['top_buy2buy_wgt']
        
    aids4_df = aids4_df.sort_values(['session', 'wgt'], ascending=[True, False], ignore_index=True).reset_index(drop=True)

    # preview_df(aids4_df,5)
    max_weight_df = aids4_df.groupby('session').wgt.max().reset_index()
    df = cudf.merge(df, max_weight_df, on='session', how='left').fillna(1)
    
    df = df.sort_values(['session', 'ts'], ascending=[True, True], ignore_index=True).reset_index(drop=True)
    
    df.wgt = df.wgt.astype('float32')
    df.wgt = df.wgt + df.repetitions
    df.wgt = df.wgt + df.ts_distance
    if for_event=='orders':
        type_weghts = [1, 10, 2]
    else:
        type_weghts = [2, 3, 1]
    for i in range(3):
        df.loc[df.type==i, 'wgt'] = df.loc[df.type==i, 'wgt'] + type_weghts[i]
    df['top_buys_wgt'] = -1
    df['top_buy2buy_wgt'] = -1
    del df['ts'], df['ts_max'], df['ts_distance']

#     aids4_df['ts'] = df.ts.max()+1
    aids4_df['type'] = -1
    aids4_df['repetitions'] = -1
    # del df['type']
    # del df['repetitions']
    
#     df = df.sort_values(['session', 'ts'], ignore_index=True).reset_index(drop=True)


    aids4_df = cudf.concat([df, aids4_df])

    aids4_df = aids4_df.sort_values(['session', 'wgt'], ascending=[True, False], ignore_index=True)
    aids4_df = aids4_df.drop_duplicates(['session', 'aid'], keep='first', ignore_index=True)
    aids4_df = aids4_df.sort_values(['session', 'wgt'], ascending=[True, False], ignore_index=True)
    aids4_df = aids4_df.reset_index(drop=True)
#     preview_df(aids4_df,2)


    aids4_df['n'] = aids4_df.groupby('session').cumcount()
    aids4_df = aids4_df.loc[aids4_df.n < keep_amount]
    del aids4_df['n']
    # preview_df(aids4_df,5)


    aids4_df.repetitions = aids4_df.repetitions.astype('int8')
    aids4_df.type = aids4_df.type.astype('int8')
    aids4_df.top_buys_wgt = aids4_df.top_buys_wgt.astype('float32')
    aids4_df.top_buy2buy_wgt = aids4_df.top_buy2buy_wgt.astype('float32')

    aids4_df.wgt = aids4_df.wgt.astype('float32')
    aids4_df = aids4_df.rename(columns={'wgt': f'{for_event}_final_wgt'})

    return aids4_df

In [26]:
def add_items_from_click_matrix(df, keep_amount=cfg.keep_amount):

    # df = df.drop_duplicates(['session', 'aid'], ignore_index=True) # other ways of duplicates handling can be done
    df['repetitions'] = 0.5
    df = cudf.merge(
        df[['session', 'aid', 'type','ts']], 
        df.groupby(['session', 'aid', 'type']).repetitions.sum().reset_index(),
        on=['session', 'aid', 'type'],
        how='inner'
    )
    
    df = cudf.merge(
        df, 
        df.groupby('session').ts.max().reset_index().rename(columns={'ts': 'ts_max'}), 
        on='session', how='inner'
    )

    df['ts_distance'] = df['ts_max'] - df['ts']
    df['ts_distance'] = np.log(df['ts_distance']+1)
    df['ts_distance'] = df['ts_distance'].max()- df['ts_distance']

#     preview_df(df)

    aids2_df = cudf.merge(df, top_20_clicks_df, left_on='aid', right_on='aid_x', how='inner')
    # preview_df(aids2_df,5)

    aids2_df = aids2_df.groupby(['session', 'aid_y']).wgt.sum().reset_index()
    aids2_df.wgt = aids2_df.wgt.astype('float32')
#     aids2_df.wgt = np.log(1+aids2_df.wgt)
    aids2_df.loc[aids2_df.wgt > 100, 'wgt'] = 100
    

    aids2_df = aids2_df.sort_values(['session', 'wgt'], ascending=[True, False], ignore_index=True).reset_index(drop=True)
    aids2_df = aids2_df.rename(columns={'aid_y': 'aid'})
    aids2_df['clicks_similarity_wgt'] = aids2_df['wgt']
    # preview_df(aids2_df,5)


    max_weight_df = aids2_df.groupby('session').wgt.max().reset_index()
    df = cudf.merge(df, max_weight_df, on='session', how='left').fillna(1)
    df.wgt = df.wgt.astype('float32')
    
    df = df.sort_values(['session', 'ts'], ignore_index=True).reset_index(drop=True)
    
    df.wgt = df.wgt + df.ts_distance
    df.wgt = df.wgt + df.repetitions
    type_weghts = [1, 3, 2]
    for i in range(3):
        df.loc[df.type==i, 'wgt'] = df.loc[df.type==i, 'wgt'] + type_weghts[i]

    del df['ts'], df['ts_max'], df['ts_distance']
    df['clicks_similarity_wgt'] = -1
    aids2_df['type'] = -1
    aids2_df['repetitions'] = -1
    aids2_df = cudf.concat([df, aids2_df])
    # preview_df(aids2_df,5)
    
    aids2_df = aids2_df.sort_values(['session', 'wgt'], ascending=[True, False], ignore_index=True)
    aids2_df = aids2_df.drop_duplicates(['session', 'aid'], ignore_index=True)
    aids2_df = aids2_df.sort_values(['session', 'wgt'], ascending=[True, False], ignore_index=True).reset_index(drop=True)
    
    aids2_df['n'] = aids2_df.groupby('session').cumcount()
    aids2_df = aids2_df.loc[aids2_df.n < keep_amount]
    del aids2_df['n']
#     preview_df(aids2_df,2)


    aids2_df.type = aids2_df.type.astype('int8')
    aids2_df.repetitions = aids2_df.repetitions.astype('int8')
    aids2_df.clicks_similarity_wgt = aids2_df.clicks_similarity_wgt.astype('float32')

    aids2_df.wgt = aids2_df.wgt.astype('float32')
    aids2_df = aids2_df.rename(columns={'wgt': f'clicks_final_wgt'})

    return aids2_df.reset_index(drop=True)

### Clicks

In [27]:
top_20_clicks_df = load_top_20_clicks_df()

top_20_clicks_v6_0.pqt top_20_clicks_v6_1.pqt top_20_clicks_v6_2.pqt top_20_clicks_v6_3.pqt 

In [28]:
uniq_test_sessions = test_df.session.unique()
uniq_test_sessions_df = cudf.DataFrame({'session': uniq_test_sessions})
chunks = 10
chunk_size = int(np.ceil(len(uniq_test_sessions) / chunks))
for i in range(chunks):
    print(i, end=' ')
    from_i = i*chunk_size
    to_i = min((i+1)*chunk_size, len(uniq_test_sessions))
    # print(mils_format(from_i),' - ', mils_format(to_i), end=' ')
    
    tmp = cudf.merge(cudf.DataFrame(test_df), uniq_test_sessions_df[from_i:to_i], on='session', how='inner')
    
    pred_df_clicks = add_items_from_click_matrix(
        cudf.DataFrame(tmp).sort_values(["session", "ts"]), 
        keep_amount=cfg.keep_amount,
    )

    if cfg.add_most_common:

        tmp_sizes = pred_df_clicks.groupby('session').size().reset_index()
        tmp_sizes.columns=['session', 'session_size']

        less_then_keep_preds = cudf.merge(pred_df_clicks, tmp_sizes.loc[tmp_sizes.session_size < cfg.keep_amount], on='session', how='inner')
        del less_then_keep_preds['session_size']
        less_then_keep_preds = add_most_common(less_then_keep_preds, top_clicks_df, event_name='clicks', keep_amount=cfg.keep_amount)

        pred_df_clicks = cudf.merge(pred_df_clicks, tmp_sizes.loc[tmp_sizes.session_size >= cfg.keep_amount], on='session', how='inner')
        pred_df_clicks = pred_df_clicks.to_pandas()
        pred_df_clicks = pd.concat([pred_df_clicks, less_then_keep_preds], ignore_index=True).sort_values('session').reset_index(drop=True)
        pred_df_clicks.to_parquet(f'pred_df_clicks_{i}.pqt')

        del tmp_sizes, less_then_keep_preds
    else:
        pred_df_clicks = pred_df_clicks.to_pandas()
        pred_df_clicks.to_parquet(f'pred_df_clicks_{i}.pqt')

    print('Predictions per session', pred_df_clicks.groupby('session').size().mean())

    pred_df_clicks = pred_df_clicks.groupby('session').aid.apply(list)
    pred_df_clicks = pred_df_clicks.add_suffix("_clicks")
    clicks_pred_df = pred_df_clicks.reset_index()
    clicks_pred_df.columns = ["session_type", "labels"]

    clicks_pred_df.to_parquet(f'clicks_pred_df_{i}.pqt')

    del pred_df_clicks, clicks_pred_df, tmp
    _ = gc.collect()

print('---' * 10)

0 

/opt/conda/lib/python3.7/site-packages/cudf/core/frame.py:2600: UserWarning: When using a sequence of booleans for `ascending`, `na_position` flag is not yet supported and defaults to treating nulls as greater than all numbers
  "When using a sequence of booleans for `ascending`, "


Predictions per session 37.56719364042565
1 Predictions per session 37.58160915415029
2 Predictions per session 37.590892505727325
3 Predictions per session 37.5406355985429
4 Predictions per session 37.5459053361327
5 Predictions per session 37.56969990608981
6 Predictions per session 37.52839736572936
7 Predictions per session 37.57095602969237
8 Predictions per session 37.56210933060575
9 Predictions per session 37.53142235036549
------------------------------


### Orders

In [29]:
# Events at same time in session:

top_20_buys_df = load_top_20_buys_df()
top_20_buy2buy_df = load_top_20_buy2buy_df()

top_15_carts_orders_v6_0.pqt top_15_carts_orders_v6_2.pqt top_15_carts_orders_v6_1.pqt top_15_carts_orders_v6_3.pqt top_15_buy2buy_v6_0.pqt 

In [30]:
uniq_test_sessions = test_df.session.unique()
uniq_test_sessions_df = cudf.DataFrame({'session': uniq_test_sessions})
chunks = 10
chunk_size = int(np.ceil(len(uniq_test_sessions) / chunks))
for i in range(chunks):
    print(i, end=' ')
    from_i = i*chunk_size
    to_i = min((i+1)*chunk_size, len(uniq_test_sessions))
    # print(mils_format(from_i),' - ', mils_format(to_i), end=' ')
    
    tmp = cudf.merge(cudf.DataFrame(test_df), uniq_test_sessions_df[from_i:to_i], on='session', how='inner')
    
    pred_df_orders = add_items_from_buys_matrices(
        cudf.DataFrame(tmp).sort_values(["session", "ts"]), 
        keep_amount=cfg.keep_amount,
        weight_const1=cfg.weight_const1_orders,
#         weight_const1=1.5,
        for_event='orders'
    )

    if cfg.add_most_common:

        tmp_sizes = pred_df_orders.groupby('session').size().reset_index()
        tmp_sizes.columns=['session', 'session_size']

        less_then_keep_preds = cudf.merge(pred_df_orders, tmp_sizes.loc[tmp_sizes.session_size < cfg.keep_amount], on='session', how='inner')
        del less_then_keep_preds['session_size']
        less_then_keep_preds = add_most_common(less_then_keep_preds, top_orders_df, event_name='orders', keep_amount=cfg.keep_amount)

        pred_df_orders = cudf.merge(pred_df_orders, tmp_sizes.loc[tmp_sizes.session_size >= cfg.keep_amount], on='session', how='inner')
        pred_df_orders = pred_df_orders.to_pandas()
        pred_df_orders = pd.concat([pred_df_orders, less_then_keep_preds], ignore_index=True).sort_values('session').reset_index(drop=True)
        pred_df_orders.to_parquet(f'pred_df_orders_{i}.pqt')

        del tmp_sizes
    else:
        pred_df_orders = pred_df_orders.to_pandas()
        pred_df_orders.to_parquet(f'pred_df_orders_{i}.pqt')

    print('Predictions per session', pred_df_orders.groupby('session').size().mean())


    pred_df_orders = pred_df_orders.groupby('session').aid.apply(list)
    pred_df_orders = pred_df_orders.add_suffix("_orders")
    orders_pred_df = pred_df_orders.reset_index()
    orders_pred_df.columns = ["session_type", "labels"]

#     preview_df(orders_pred_df)
    # print('Orders_pred len:', len_mils(orders_pred_df))
    orders_pred_df.to_parquet(f'orders_pred_df_{i}.pqt')

    del pred_df_orders, orders_pred_df, tmp
    _ = gc.collect()

print('---' * 10)

0 Predictions per session 39.424001531274484
1 Predictions per session 39.434385486389004
2 Predictions per session 39.4403311381078
3 Predictions per session 39.4301804630909
4 Predictions per session 39.419892212631815
5 Predictions per session 39.422548016820095
6 Predictions per session 39.44074386443435
7 Predictions per session 39.43743009073998
8 Predictions per session 39.44344752095035
9 Predictions per session 39.4251857346238
------------------------------


## Carts

In [31]:
try:
    print(len_mils(top_20_buys_df))
except:
    top_20_buys_df = load_top_20_buys_df()
    print(len_mils(top_20_buys_df))
try:
    print(len_mils(top_click2cart_df))
except:
    top_click2cart_df = load_top_click2cart_df()
    print(len_mils(top_click2cart_df))

52,375,804
56,790,872


In [32]:
uniq_test_sessions = test_df.session.unique()
uniq_test_sessions_df = cudf.DataFrame({'session': uniq_test_sessions})
chunks = 10
chunk_size = int(np.ceil(len(uniq_test_sessions) / chunks))
for i in range(chunks):
    print(i, end=' ')
    from_i = i*chunk_size
    to_i = min((i+1)*chunk_size, len(uniq_test_sessions))
    
    tmp = cudf.merge(cudf.DataFrame(test_df), uniq_test_sessions_df[from_i:to_i], on='session', how='inner')
    
    pred_df_carts = add_items_from_buys_matrices(
        cudf.DataFrame(tmp).sort_values(["session", "ts"]), 
        keep_amount=cfg.keep_amount,
        weight_const1=cfg.weight_const1_carts,
#         weight_const1=0.1,
        for_event='carts'
    )
    
    if cfg.add_most_common:
        tmp_sizes = pred_df_carts.groupby('session').size().reset_index()
        tmp_sizes.columns=['session', 'session_size']

        less_then_keep_preds = cudf.merge(pred_df_carts, tmp_sizes.loc[tmp_sizes.session_size < cfg.keep_amount], on='session', how='inner')
        del less_then_keep_preds['session_size']
        less_then_keep_preds = add_most_common(less_then_keep_preds, top_carts_df, event_name='carts', keep_amount=cfg.keep_amount)

        pred_df_carts = cudf.merge(pred_df_carts, tmp_sizes.loc[tmp_sizes.session_size >= cfg.keep_amount], on='session', how='inner')
        pred_df_carts = pred_df_carts.to_pandas()
        pred_df_carts = pd.concat([pred_df_carts, less_then_keep_preds], ignore_index=True).sort_values('session').reset_index(drop=True)
        pred_df_carts.to_parquet(f'pred_df_carts_{i}.pqt')

        del tmp_sizes, less_then_keep_preds
    else:
        pred_df_carts = pred_df_carts.to_pandas()
        pred_df_carts.to_parquet(f'pred_df_carts_{i}.pqt')

    print('Predictions per session', pred_df_carts.groupby('session').size().mean())

    pred_df_carts = pred_df_carts.groupby('session').aid.apply(list)
    pred_df_carts = pred_df_carts.add_suffix("_carts")
    carts_pred_df = pred_df_carts.reset_index()
    carts_pred_df.columns = ["session_type", "labels"]
    
    carts_pred_df.to_parquet(f'carts_pred_df_{i}.pqt')

    del pred_df_carts, carts_pred_df, tmp
    _ = gc.collect()

print('---' * 10)

0 Predictions per session 39.67310280474456
1 Predictions per session 39.65627074847022
2 Predictions per session 39.657927635317414
3 Predictions per session 39.655881948307524
4 Predictions per session 39.66097223966838
5 Predictions per session 39.65311249484092
6 Predictions per session 39.65584605906174
7 Predictions per session 39.65559483434122
8 Predictions per session 39.66975912334536
9 Predictions per session 39.65500616124517
------------------------------


# Create submission 

In [33]:
ev_preds_df = {}
events_list = ['clicks', 'carts', 'orders']
for ev in events_list:
    files_for_ev = glob.glob(f'{ev}_pred_df*')
    print(ev, len(files_for_ev))
    ev_preds_df[ev] = pd.DataFrame()
    for f in files_for_ev:
        ev_preds_df[ev] = pd.concat([ev_preds_df[ev], pd.read_parquet(f)], ignore_index=True)
    preview_df(ev_preds_df[ev])

print('Submission: ')
pred_df = pd.concat([ev_preds_df[ev] for ev in events_list], ignore_index=True).reset_index(drop=True)
pred_df["labels"] = pred_df.labels.apply(lambda x: " ".join(map(str,x)))
pred_df.to_csv("submission.csv", index=False)
preview_df(pred_df,5)

clicks 10
1,671,803


,session_type,labels
0,13468503_clicks,"[1354951, 1685214, 1531805, 1833893, 1208112, ..."


carts 10
1,671,803


,session_type,labels
0,13468503_carts,"[1354951, 1685214, 1531805, 1833893, 1208112, ..."


orders 10
1,671,803


,session_type,labels
0,13099779_orders,"[972319, 245308, 1100749, 1090801, 128649, 161..."


Submission: 
5,015,409


,session_type,labels
0,13468503_clicks,1354951 1685214 1531805 1833893 1208112 8557 1...
1,13468504_clicks,239165 349457 1610340 1675335 245654 945619 41...
2,13468505_clicks,26107 558556 9282 242470 1783897 1411330 15542...
3,13468506_clicks,1124416 1695413 1667230 990186 380647 1533580 ...
4,13468507_clicks,1305113 604767 336884 1012794 510211 1057728 1...
